# DC offset and Signal-to-Noise Ratio (SNR) estimation

In [1]:
import sys
sys.path.append('../')

import torch
import csv

from pathlib import Path

In [2]:
DATA_DIR = Path('../data/raw/')
MODELS_DIR = Path('../models/')
RESULTS_DIR = Path('../data/features/')
PLOTS_DIR = Path('../docs/plots/')

## DC offset

\begin{equation}
\text{DC Offset} = \frac{1}{N} \sum_{i=1}^{N} x_i
\end{equation}

In [3]:
def compute_dc_offset(signal):
    """
    Compute the DC offset of an audio signal.
    """
    return torch.mean(signal)

## Signal-to-Noise Ratio

\begin{equation}
\text{res} = \text{dry} - \text{wet}
\end{equation}

\begin{equation}
\text{SNR (dB)} = 10 \cdot \log_{10}\left(\frac{\text{wet}^2}{\text{res}^2 + \text{eps}}\right)
\end{equation}

where $\text{wet}$ is the wet signal, $\text{res}$ is the residual signal, and $\text{eps}$ is a small number to avoid division by zero.

In [4]:
def compute_snr(dry, wet, eps=1e-8):
    """
    Compute the Signal-to-Noise Ratio (SNR) using PyTorch.
    Values are in dB.
    """
    # reduce potential DC offset
    dry = dry - dry.mean()
    wet = wet - wet.mean()

    # compute SNR
    res = dry - wet
    snr = 10 * torch.log10(
        (wet ** 2).sum() / ((res ** 2).sum() + eps)
    )
    return snr.item()